In [ ]:
import pandas as pd
# read the dataset in from the excel file
df = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
# show the first few rows
df.head()

In [ ]:
df['article_id'] = df['article_id'].apply(lambda x: '_'+str(x))

# DataFrame for building the recommendation system
orders = pd.DataFrame(columns=['customer_id', 'article_id'])

customer_id = []
article_id = []

for c in df['customer_id'].unique():
    customer_id.append(c)
    article_id.append(df[df['customer_id'] == c]['article_id'])

orders['customer_id'] = customer_id
orders['article_id'] = article_id

# DataFrame for retrieving product descriptions
articles = df[['article_id']].copy()

articles = articles[~articles.duplicated()]

In [ ]:
orders.head()

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()

orders_1hot = te.fit_transform(orders['article_id'])

In [ ]:
# convert orders_1hot to a DataFrame
orders_1hot = pd.DataFrame(orders_1hot, columns =te.columns_)
# Inspect the first 5 rows of the DataFrame
orders_1hot.head()

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
# the moment we have all been waiting for (again)
orders_fp = fpgrowth(orders_1hot, min_support=0.01, max_len=2,
                use_colnames=True)


In [ ]:
from mlxtend.frequent_patterns import association_rules
ar = association_rules(is_fp, 
                       metric="lift", 
                       min_threshold=0.01)

In [ ]:
def predict(antecedent, rules, max_results= 12):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results]

In [ ]:
preds = predict({df['article_id'][0]}, rules)
preds